## 📂 Pivot Table

Pivot Table이란 **가지고 있는 데이터 원본을 원하는 형태의 가공된 정보로 보여주는 것**이며 자료의 형태를 변경하기 위해 많이 사용하는 방법이다.<br>
Pivot Table은 두 개의 키를 사용해서 데이터를 선택하며 자세한 사용법은 다음과 같다.<br>

**pivot_table(data, index = '행 위치에 들어갈 열', columns = '열 위치에 들어갈 열, values = '데이터로 사용할 열', aggfunc = '데이터 집계함수')**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
data = {
    '도시':['서울', '서울', '서울', '부산', '부산', '부산', '인천', '인천'],
    '연도':['2015', '2010', '2005', '2015', '2010', '2005', '2015', '2010'],
    '인구':[9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    '지역':['수도권', '수도권', '수도권', '경상권', '경상권', '경상권', '수도권', '수도권']
}

columns = ['도시', '연도', '인구', '지역']
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


아래는 각 도시에 대한 연도별 인구 수를 구하는 예제이다.

In [4]:
pd.pivot_table(df1, index='도시', columns='연도', values='인구')

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


아래는 각 지역별 도시에 대한 연도별 인구 수를 구하는 예제이다.

In [5]:
pd.pivot_table(df1, index=['지역', '도시'], columns='연도', values='인구')

연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

In [6]:
df2 = sns.load_dataset('titanic')[['age', 'sex', 'class', 'fare', 'survived']]
df2.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


아래는 각 선실 등급별로 숙박객의 성별별 나이의 평균을 구하는 예제이다.

In [7]:
pd.pivot_table(df2,
               index='class',
               columns='sex',
               values='age',
               aggfunc='mean')

sex,female,male
class,,
First,34.611765,41.281386
Second,28.722973,30.740707
Third,21.750000,26.507589


아래는 각 선실 등급별로 숙박객의 성별별 나이의 평균과 합계를 구하는 예제이다.

In [8]:
pd.pivot_table(df2,
               index='class',
               columns='sex',
               values='age',
               aggfunc=['mean', 'sum'])

mean                sum         
sex        female       male  female     male
class                                        
First   34.611765  41.281386  2942.0  4169.42
Second  28.722973  30.740707  2125.5  3043.33
Third   21.750000  26.507589  2218.5  6706.42

아래는 선실 등급, 성별, 생존여부별 나이와 티켓값의 평균, 최대값을 구하는 예제이다.

In [9]:
pd.pivot_table(df2,
               index=['class', 'sex'],
               columns='survived',
               values=['age', 'fare'],
               aggfunc=['mean', 'max'])

mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   
       male    27.255814  22.274211   12.204469   15.579696  74.0  45.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875  
       male     69.55   56.4958

---

## 📂 그룹 분석

만약 키가 지정하는 조건에 맞는 데이터가 하나 이상이라서 데이터 그룹을 이루는 경우에는 그룹의 특성을 보여주는 그룹분석을 해야 한다.<br>
그룹 분석은 Pivot Table과 달리 키에 의해서 결정되는 데이터가 여러개 있을 경우 미리 지정한 연산을 통해 그 그룹 데이터의 대표값을 계산하는 것이다. 

In [10]:
np.random.seed(0)
df3 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df3

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


우선 분석하고자 하는 시리즈나 데이터프레임에 groupby 메서드를 호출하여 그룹화 수행한다.

In [11]:
groups = df3.groupby(df3.key1)
groups

In [12]:
groups.groups

{'A': [0, 1, 4], 'B': [2, 3]}

In [13]:
pd.DataFrame(groups)

,0,1
0,A,key1 key2 data1 data2 0 A one 1 ...
1,B,key1 key2 data1 data2 2 B one 3 ...


In [14]:
pd.DataFrame(groups).loc[0].values

array(['A',   key1 key2  data1  data2
            0    A  one      1     10
            1    A  two      2     20
            4    A  one      5     50], dtype=object)

In [15]:
pd.DataFrame(groups).loc[1].values

array(['B',   key1 key2  data1  data2
            2    B  one      3     30
            3    B  two      4     40], dtype=object)

대표적인 그룹 연산 함수의 종류는 아래와 같다.
- size, count : 그룹 데이터의 개수
- mean, median, min, max: 그룹 데이터의 평균, 중앙값, 최소, 최대
- sum, prod, std, var, quantile : 그룹 데이터의 합계, 곱, 표준편차, 분산, 사분위수
- first, last: 그룹 데이터 중 첫 번째 데이터와 마지막 데이터
- describe : 대표적인 통계치 계산

아래와 같이 그룹 객체에 대해 그룹 연산을 수행한다.

In [16]:
groups.sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [17]:
groups['data1'].sum()
groups.sum()['data1']

key1
A    8
B    7
Name: data1, dtype: int64

key1
A    8
B    7
Name: data1, dtype: int64

In [18]:
groups[['data1']].sum()
groups.sum()[['data1']]

,data1
key1,
A,8
B,7


,data1
key1,
A,8
B,7


agg(), apply()는 원하는 그룹 연산이 없는 경우 사용자 정의 함수를 그룹 객체에 적용할 수 있도록 하는 함수이다.

In [19]:
iris = sns.load_dataset('iris')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [20]:
i_groups = iris.groupby(iris.species)

In [21]:
i_groups.sum()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,250.3,171.4,73.1,12.3
versicolor,296.8,138.5,213.0,66.3
virginica,329.4,148.7,277.6,101.3


In [22]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()

In [23]:
i_groups.agg(peak_to_peak_ratio)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


In [24]:
def top3_petal_length(df):
    return df.sort_values(by='petal_length', ascending=False)[:3]

In [25]:
i_groups.apply(top3_petal_length)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

아래는 species로 분류된 각 그룹에 대해 petal_length를 소, 중, 대 3 구간으로 분류한 예제이다.

In [26]:
def q3cut(s):
    return pd.qcut(s, 3, labels=['소', '중', '대']).astype(str)

In [27]:
iris.groupby(iris.species).petal_length.apply(q3cut)

0      소
1      소
2      소
3      중
4      소
      ..
145    소
146    소
147    소
148    중
149    소
Name: petal_length, Length: 150, dtype: object

In [28]:
iris['petal_length_class'] = iris.groupby(iris.species).petal_length.apply(q3cut)
iris

,sepal_length,sepal_width,petal_length,petal_width,species,petal_length_class
0,5.1,3.5,1.4,0.2,setosa,소
1,4.9,3.0,1.4,0.2,setosa,소
2,4.7,3.2,1.3,0.2,setosa,소
3,4.6,3.1,1.5,0.2,setosa,중
4,5.0,3.6,1.4,0.2,setosa,소
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,소
146,6.3,2.5,5.0,1.9,virginica,소
147,6.5,3.0,5.2,2.0,virginica,소
148,6.2,3.4,5.4,2.3,virginica,중


---

## 📂 Pivot Table · 그룹 함수 예제

In [29]:
tips = sns.load_dataset('tips')
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [30]:
tips['tip_pt'] = tips['tip'] / tips['total_bill']
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pt
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [31]:
tips.groupby(['sex', 'smoker']).size() # 흡연 유무에 따른 성별별 인원

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [32]:
tips.groupby('sex')['tip_pt'].mean() # 여성이 식사 금액 대비 팁의 비율이 더 높다

sex
Male      0.157651
Female    0.166491
Name: tip_pt, dtype: float64

In [33]:
tips.groupby('smoker')['tip_pt'].mean() # 흡연 유무에 따른 팁 비율의 평균

smoker
Yes    0.163196
No     0.159328
Name: tip_pt, dtype: float64

In [34]:
tips.pivot_table('tip_pt',index='sex', columns='smoker') # 흡연자가 비흡연자에 비해 팁 비율이 높다

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


In [35]:
tips.groupby(['sex', 'smoker'])[['tip_pt']].describe() # 여성이면서 흡연자가 팁 비율이 가장 높다

tip_pt                                                    \
               count      mean       std       min       25%       50%   
sex    smoker                                                            
Male   Yes      60.0  0.152771  0.090588  0.035638  0.101845  0.141015   
       No       97.0  0.160669  0.041849  0.071804  0.131810  0.157604   
Female Yes      33.0  0.182150  0.071595  0.056433  0.152439  0.173913   
       No       54.0  0.156921  0.036421  0.056797  0.139708  0.149691   

                                   
                    75%       max  
sex    smoker                      
Male   Yes     0.191697  0.710345  
       No      0.186220  0.291990  
Female Yes     0.198216  0.416667  
       No      0.181630  0.252672